In [1]:
DATA_DIR = 'data/'
RESULT_DIR = 'results/original_vs_updated/'
import os
# Create the directories
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(RESULT_DIR, exist_ok=True)

In [2]:
import pandas as pd 
import wandb
api = wandb.Api()

# Project is specified by <entity/project-name>
runs = api.runs("dl2-2024/dl-2024")

summary_list, config_list, name_list = [], [], []
for run in runs: 
    # .summary contains the output keys/values for metrics like accuracy.
    #  We call ._json_dict to omit large files 
    summary_list.append(run.summary._json_dict)

    # .config contains the hyperparameters.
    #  We remove special values that start with _.
    config_list.append(
        {k: v for k,v in run.config.items()
          if not k.startswith('_')})

    # .name is the human-readable name of the run.
    name_list.append(run.name)

runs_df = pd.DataFrame({
    "summary": summary_list,
    "config": config_list,
    "name": name_list
    })

runs_df.to_csv(DATA_DIR + "project.csv")

In [3]:
import pandas as pd
from tabulate import tabulate
import numpy as np

# Read the CSV file
runs_df = pd.read_csv(DATA_DIR + "project.csv")

# Extract and flatten the summary and config columns
summary_df = pd.json_normalize(runs_df['summary'].apply(eval))
config_df = pd.json_normalize(runs_df['config'].apply(eval))

# Combine the flattened summary and config data
results_df = pd.concat([runs_df[['name']], config_df, summary_df], axis=1)

# Filter results based on `softmax` and `use_underscore` settings
original_code = results_df[(results_df['softmax'] == True) & (results_df['use_underscore'] == True) & (results_df['full_val'] != True)]
updated_code = results_df[(results_df['softmax'] == False) & (results_df['use_underscore'] == False) & (results_df['full_val'] != True)]

# Function to get the specified metric for each dataset, model, and transformation combination
def get_metric(df, dataset_name, model_name, transformation, metric):
    result = df[(df['dataset_name'] == dataset_name) & 
                (df['model_name'] == model_name) & 
                (df['data_transformations'] == transformation)][metric]
    return np.round(result.iloc[0], 2) if not result.empty else None

# Create a summary DataFrame for each combination
architectures = ['RN50']  # Add more architectures if needed
transformations = ['flip', 'rot90']
datasets = ['CIFAR100', 'ISIC2018']

# Initialize the summary data structure
summary_data = {
    "Architecture-Transformation": []
}

# Initialize columns for each dataset and metric
metrics = ["val_top1_acc", "final_top1_acc"]
for dataset in datasets:
    for metric in metrics:
        summary_data[f"{dataset} Original {metric.replace('_', ' ').title()}"] = []
        summary_data[f"{dataset} Updated {metric.replace('_', ' ').title()}"] = []

# Populate the summary data
for architecture in architectures:
    for transformation in transformations:
        combination = f"{architecture} {transformation}"
        summary_data["Architecture-Transformation"].append(combination)
        for dataset in datasets:
            for metric in metrics:
                summary_data[f"{dataset} Original {metric.replace('_', ' ').title()}"].append(
                    get_metric(original_code, dataset, architecture, transformation, metric))
                summary_data[f"{dataset} Updated {metric.replace('_', ' ').title()}"].append(
                    get_metric(updated_code, dataset, architecture, transformation, metric))

summary_df = pd.DataFrame(summary_data)

# Save the summary table to a new CSV file
# summary_df.to_csv(RESULT_DIR + "final_accuracy_summary.csv", index=False)

# Convert the dataframe to a markdown table
markdown_table = tabulate(summary_df, headers='keys', tablefmt='pipe')

# Save the markdown table to a file
with open(RESULT_DIR + "original_vs_updated_summary.md", "w") as file:
    file.write(markdown_table)

# Display the markdown table
print(markdown_table)


|    | Architecture-Transformation   |   CIFAR100 Original Val Top1 Acc |   CIFAR100 Updated Val Top1 Acc |   CIFAR100 Original Final Top1 Acc |   CIFAR100 Updated Final Top1 Acc |   ISIC2018 Original Val Top1 Acc |   ISIC2018 Updated Val Top1 Acc |   ISIC2018 Original Final Top1 Acc |   ISIC2018 Updated Final Top1 Acc |
|---:|:------------------------------|---------------------------------:|--------------------------------:|-----------------------------------:|----------------------------------:|---------------------------------:|--------------------------------:|-----------------------------------:|----------------------------------:|
|  0 | RN50 flip                     |                            36.62 |                           36.88 |                              51.97 |                             56.32 |                            44.56 |                           15.54 |                              44.56 |                             53.89 |
|  1 | RN50 rot90              

In [4]:
datasets = ['CIFAR100']
metrics = ["prefinetune_top1_acc", "val_top1_acc", "finetune_top1_acc",  "final_top1_acc"]
transformations = ['rot90']

# Do the same for the full_val setting
original_code_full_val = results_df[(results_df['softmax'] == True) & (results_df['use_underscore'] == True) & (results_df['full_val'] == True)]
updated_code_full_val = results_df[(results_df['softmax'] == False) & (results_df['use_underscore'] == False) & (results_df['full_val'] == True)]

summary_data_full_val = {
    "Architecture-Transformation": []
}

for dataset in datasets:
    for metric in metrics:
        summary_data_full_val[f"{dataset} Original {metric.replace('_', ' ').title()}"] = []
        summary_data_full_val[f"{dataset} Updated {metric.replace('_', ' ').title()}"] = []


for dataset in datasets:
    for metric in metrics:
        summary_data_full_val[f"{dataset} Original {metric.replace('_', ' ').title()}"] = []
        summary_data_full_val[f"{dataset} Updated {metric.replace('_', ' ').title()}"] = []

# Populate the summary data
for architecture in architectures:
    for transformation in transformations:
        combination = f"{architecture} {transformation}"
        summary_data_full_val["Architecture-Transformation"].append(combination)
        for dataset in datasets:
            for metric in metrics:
                summary_data_full_val[f"{dataset} Original {metric.replace('_', ' ').title()}"].append(
                    get_metric(original_code_full_val, dataset, architecture, transformation, metric))
                summary_data_full_val[f"{dataset} Updated {metric.replace('_', ' ').title()}"].append(
                    get_metric(updated_code_full_val, dataset, architecture, transformation, metric))
                
summary_df_full_val = pd.DataFrame(summary_data_full_val)

# Save the summary table to a new CSV file
# summary_df_full_val.to_csv(RESULT_DIR + "full_val_accuracy_summary.csv", index=False)

# Convert the dataframe to a markdown table
markdown_table_full_val = tabulate(summary_df_full_val, headers='keys', tablefmt='pipe')

# Save the markdown table to a file
with open(RESULT_DIR + "original_vs_updated_full_val_summary.md", "w") as file:
    file.write(markdown_table_full_val)

# Add a caption to the markdown table
markdown_table_full_val = f"## Original vs Updated Full Val Summary\n{markdown_table_full_val}"
# Display the markdown table
print(markdown_table_full_val)

## Original vs Updated Full Val Summary
|    | Architecture-Transformation   |   CIFAR100 Original Prefinetune Top1 Acc |   CIFAR100 Updated Prefinetune Top1 Acc |   CIFAR100 Original Val Top1 Acc |   CIFAR100 Updated Val Top1 Acc |   CIFAR100 Original Finetune Top1 Acc |   CIFAR100 Updated Finetune Top1 Acc |   CIFAR100 Original Final Top1 Acc |   CIFAR100 Updated Final Top1 Acc |
|---:|:------------------------------|-----------------------------------------:|----------------------------------------:|---------------------------------:|--------------------------------:|--------------------------------------:|-------------------------------------:|-----------------------------------:|----------------------------------:|
|  0 | RN50 rot90                    |                                    26.08 |                                    35.2 |                            31.63 |                           34.76 |                                 52.78 |                                53.85 